In [1]:
%run Reddit_import_and_functions.ipynb

pd.set_option('display.max_colwidth',350)

In [2]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

from sklearn.naive_bayes import MultinomialNB # ComplementNB, not available in sklearn < 0.2
from sklearn import svm
from sklearn.externals import joblib

from sklearn.preprocessing import RobustScaler

C:\Users\Jason\AppData\Local\conda\conda\envs\DSI-6\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [12]:
dfraw = pd.read_csv('Train1allinfo.csv')

## Check Spam Filter

In [21]:
def checkspam(df) :
    mask = []
    for index in df.index :
        corpus = df.loc[index, 'corpus']
        if 'remove' in corpus : mask.append(True)
        elif 'delete' in corpus :
            mask.append(True)
            print(len(mask))
        elif df.loc[index, 'rowsum'] == 0 : mask.append(True)
        elif (len(splitText(corpus)) / df.loc[index, 'rowsum']) > 2 :
            mask.append(True)
            print(len(mask))
        else : mask.append(False)
            
    return mask

In [22]:
with h5py.File("masterdata.hdf5",'r') as f :
    rowsumshalf0 = f['rowsums/rowsumshalf0']
    rowsumshalf0 = rowsumshalf0[:]
    rowsumshalf1 = f['rowsums/rowsumshalf1']
    rowsumshalf1 = rowsumshalf1[:]
    print(rowsumshalf0.shape,rowsumshalf1.shape)

(220073,) (220072,)


In [23]:
dfraw['rowsum'] = np.concatenate((rowsumshalf0, rowsumshalf1))

In [17]:
dfraw[dfraw['author']=='[deleted]']

,Unnamed: 0,index,datetime,corpus,permalink,author,subreddit,rowsum
16,196724,196724,1.519892e+09,cardano foundation release research threat qu...,https://www.reddit.com/r/CryptoCurrency/commen...,[deleted],0,18
17,438147,47003,1.486239e+09,france pave mile road solar panel smithsonian...,/r/environment/comments/5s19gn/france_is_pavin...,[deleted],1,15
20,304589,304589,1.514343e+09,beginner advice beginner advice,https://www.reddit.com/r/CryptoCurrency/commen...,[deleted],0,4
50,522002,130858,1.379434e+09,factory food satellite image industrial farm ...,/r/environment/comments/1mkdv4/factory_food_fr...,[deleted],1,11
88,424701,33557,1.504323e+09,truth harvey climate change middle washington...,/r/environment/comments/6xigo2/the_truth_about...,[deleted],1,15
91,201334,201334,1.519519e+09,countdown cme bitcoin future expiration time ...,https://www.reddit.com/r/CryptoCurrency/commen...,[deleted],0,8
112,494687,103543,1.413354e+09,need watch right youtu dqo,/r/environment/comments/2ja5xl/this_is_somethi...,[deleted],1,4
134,538111,146967,1.359185e+09,network alienate viewer leave cliff hanger ha...,/r/environment/comments/17aknv/tv_networks_ali...,[deleted],1,11
152,555945,164801,1.334209e+09,continue shot prize tusk material end sale ch...,/r/environment/comments/s5cwz/continue_to_be_s...,[deleted],1,11
159,530830,139686,1.367972e+09,nestlé stop try patent natural cure action su...,/r/environment/comments/1dvt7b/nestlé_stop_try...,[deleted],1,10


In [24]:
mask = checkspam(dfraw)

449
506
561
792
892
1071
1112
1339
1568
1609
1820
1837
1939
2086
2087
2794
3419
3563
3718
4137
4175
4334
4339
4448
4467
4700
4854
5178
5196
5339
5406
5822
5903
6077
6369
6540
6644
6710
6720
7181
7627
7805
7886
8072
8078
8099
8643
8647
8968
9345
9573
10066
10367
10720
10757
11459
11595
11665
11770
11841
12021
12031
12182
12944
13228
13306
14058
14159
14235
14250
14348
14454
14690
14833
14893
15223
15445
15455
16046
16370
16392
16585
16681
16736
16890
16994
17079
17551
17569
17597
17916
18400
18478
18566
18638
18691
18803
19204
19327
19365
19422
19660
20071
20255
20339
20420
20462
20818
20824
21105
21124
21210
21594
21630
21704
22053
22085
22162
22413
22436
22522
22548
22833
23451
23617
24353
24576
24888
24997
25200
25359
25544
26299
26482
26661
26665
26701
26912
26928
27341
27354
27715
27910
28059
28170
28693
28825
29016
29038
29351
29461
29480
29925
29974
30132
30470
30512
30551
30683
30729
30749
30848
31155
31799
31838
31919
31987
32703
33074
33137
33796
33993
34023
34178
34195
34213


259334
259484
259760
259767
259845
259989
260087
260308
260390
260600
260615
260696
260978
261177
261385
261439
262022
262111
262444
262792
263054
263477
263970
264041
264258
265131
265252
265412
265771
265998
266110
266659
267230
267792
268132
268516
268588
268619
269156
269698
269944
269950
270233
270281
270408
271491
271564
271649
271722
271863
271885
271951
272061
272147
272166
272234
272446
272666
272974
273768
273835
273935
274046
274090
274219
274300
274441
274713
274737
274767
274896
274903
275175
275235
275478
275746
275749
275771
275897
275998
276008
276013
276032
276059
276239
276330
276381
276424
276450
276471
277165
277193
277330
277397
277601
277638
277652
277773
277820
277988
278008
278813
278840
278872
279395
279483
279615
279839
279904
279924
280225
280262
280410
280487
280563
280765
280809
280976
281235
281533
281785
281831
281894
281958
282095
282410
283451
283736
284312
284486
284643
284784
284879
285107
285343
285496
285819
286026
286692
287047
287162
287201
287365

In [28]:
dfraw[dfraw['author']=='[deleted]'].shape

(35473, 8)

## Count Vectorize with No Spam Dataset

In [26]:
possiblespam = dfraw[mask].loc[:,['corpus','permalink','rowsum']]
possiblespam.head()

,corpus,permalink,rowsum
2,kraken have list tezos remove kraken have list tezos,https://www.reddit.com/r/CryptoCurrency/comments/9otuj4/kraken_has_listed_tezos/,9
4,crypto fall board knew be bubble stock market fall point day just self correct happens time people dense remove vljxi crypto fall board knew be,https://www.reddit.com/r/CryptoCurrency/comments/7vljxi/crypto_falls_across_the_board_see_i_knew_it_was_a/,22
13,irregular voting activity remove irregular voting activity,https://www.reddit.com/r/CryptoCurrency/comments/a64s7j/irregular_voting_activity/,7
15,new project horizon bitcoin fork btcalol remove jil new project horizon bitcoin fork,https://www.reddit.com/r/CryptoCurrency/comments/8k0jil/new_project_on_the_horizon_another_bitcoin_fork/,11
24,mcafee call exchange remove dqaww mcafee call exchange,https://www.reddit.com/r/CryptoCurrency/comments/9dqaww/mcafee_calls_out_exchange/,7


In [27]:
dfraw.drop(possiblespam.index, axis=0, inplace=True)
dfraw.shape

(380084, 8)

In [29]:
dfraw.drop(dfraw[dfraw['author']=='[deleted]'].index, axis=0, inplace=True)
dfraw.shape

(344611, 8)

In [34]:
y_train2 = dfraw['subreddit']
y_train2.shape

(344611,)

In [35]:
regexcondition = r'\b[a-zA-Z]{3,30}\b' # Tokens must be below 31 letters and more than 3 (checking again)
vect = CountVectorizer(dtype='uint16',min_df=40, token_pattern=regexcondition, 
                       strip_accents='ascii', decode_error='ignore', lowercase=True) 


In [36]:
X_text = vect.fit_transform(dfraw['corpus'])


In [44]:
lis = ['features']
lis.extend(vect.get_feature_names())
len(lis)

9467

In [45]:
pd.Series(lis).to_csv('masterFeaturesVector2.csv')

In [38]:
dfvect = pd.DataFrame(X_text.toarray(),dtype='uint16',columns=vect.get_feature_names(),index=dfraw.index)
dfvect.shape

(344611, 9466)

In [51]:
with h5py.File("masterdata.hdf5", "a") as z:
    dset = z.create_dataset("X_Train2", data=dfvect, dtype='i2', compression="lzf", shuffle=True)
    print(dset.shape)
    dset = z.create_dataset("y_train2", data=y_train2)

(344611, 9466)


In [46]:
dfvect.head()

,aaa,aaabead,aaron,aautomoderator,abandon,abbott,abc,abcnews,ability,able,aboard,abolish,abra,abroad,abrupt,absence,absolute,absolutely,absorb,absorbed,abstract,absurd,abu,abundance,abundant,abuse,abyss,academic,academy,accelerate,accelerates,acceleration,accelerator,accenture,accept,acceptable,acceptance,accepted,accepts,access,accessibility,accessible,accessory,accident,accidental,...,youre,youth,youtu,youtube,youtuber,youtubers,youtubetw,youve,yuan,zaif,zap,zcash,zcl,zclassic,zcoin,zdnet,zealand,zebpay,zec,zen,zencash,zendesk,zero,zerocoin,zerocrypted,zerohedge,zhang,zhao,zhu,zika,zil,zilliqa,zimbabwe,zinke,zip,zoin,zombie,zone,zoo,zoom,zrx,zuckerberg,zug,zurich,zycrypto
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [52]:
with h5py.File("masterdata.hdf5",'a') as f :
    r = f['rowsums']
    dset = r.create_dataset('rowsumsTrain2', data=dfvect.T.sum())
    print(dset.shape)
    dset = r.create_dataset('columnsumsTrain2', data=dfvect.sum())
    print(dset.shape)

(344611,)
(9466,)


In [53]:
list_all_hdf5('masterdata.hdf5')

Train1Ytrain
Train1dset
Train2dset
X_Train2
columnsums
columnsums/columnindextodrop
columnsums/columnsumshalf0
columnsums/columnsumshalf1
environment
rowsums
rowsums/columnsumsCrypto
rowsums/columnsumsEnv
rowsums/columnsumsTrain2
rowsums/crypto_dset
rowsums/env_dset
rowsums/rowsumsCrypto
rowsums/rowsumsEnv
rowsums/rowsumsTrain2
rowsums/rowsumshalf0
rowsums/rowsumshalf1
y_train2


## Random Forest with No Spam Dataset

In [4]:
with h5py.File("masterdata.hdf5", "r") as z:
    dset = z["X_Train2"]
    X_train2 = dset[0:220073]
    print(X_train2.shape)
    dset = z["y_train2"]
    y_train2 = dset[0:220073]
    print(y_train2.shape)

(220073, 9466)
(220073,)


In [6]:
rf = RandomForestClassifier(criterion='gini', verbose=1, n_jobs = 2)

In [7]:
rf_params = {
    'n_estimators': [40],
    'max_depth': [250, 350], # after: None
    'min_samples_split': [14],
    'max_features' :['auto']
}
gs = GridSearchCV(rf, param_grid=rf_params, cv=3, verbose=1)
gs.fit(df, y_train2)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed: 10.8min finished
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed:    4.0s finished
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed:    9.1s finished
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed: 10.7min finished
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed:    3.7s finished
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed:    8.6s finished
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed: 10.5min finished
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed:    3.4s finished
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed:    8.3s finished
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed: 10.9min finished
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed:    3.9s finished
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed:    9.5s finished
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed: 11.0min finished
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed:    3.8s finished
[Parallel(n_jobs=2)]

GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=2,
            oob_score=False, random_state=None, verbose=1,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [40], 'max_depth': [250, 350], 'min_samples_split': [14], 'max_features': ['auto']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [8]:
print(gs.best_score_)
print(gs.best_params_)

0.9757444120814457
{'max_depth': 350, 'max_features': 'auto', 'min_samples_split': 14, 'n_estimators': 40}


In [10]:
pred_minus_actual = pd.read_csv('pred_minus_actual.csv')
pred_minus_actual.shape

(220072, 9)

In [33]:
pred_minus_actual.drop('index', axis=1, inplace=True)

In [35]:
pred_minus_actual.columns = ['index','original_index','datetime', 'corpus', 'permalink', 'author','subreddit','rf1_220073_330108']

In [ ]:
dfraw.drop('index', axis=1, inplace=True)

In [40]:
dfraw.columns = ['index','datetime', 'corpus', 'permalink', 'author','subreddit','rowsum']
dfraw.head()

,index,datetime,corpus,permalink,author,subreddit,rowsum
0,439649,1.484256e+09,official say percent california drought powerful storm news news world official say percent california drought powerful storm,/r/environment/comments/5nk84s/officials_say_more_than_40_percent_of_california/,helpmeredditimbored,1,17
1,125153,1.527789e+09,opening door everyday user introduce iconomi fiat gateway medium iconominet iconomi fiat gateway,https://www.reddit.com/r/CryptoCurrency/comments/8nim9k/opening_the_doors_for_everyday_users_introducing/,TheAfricaBug,0,13
3,568688,1.315469e+09,free chimp cage year inject hiv cruel experiment finally feel sun face dailymail news article lab chimp freed cage fior year inject hiv,/r/environment/comments/k8g39/free_at_last_chimps_caged_for_30_years_and/,Kranky_Old_Dude,1,22
5,315118,1.513717e+09,chad satoshi imgur ssbnc,https://www.reddit.com/r/CryptoCurrency/comments/7koez1/chad_satoshi/,Brian_K9,0,3
6,102199,1.530581e+09,vechainthor mainnet launch gain backing lbank ambcrypto vechainthor ven mainnet launch gain backing lbank,https://www.reddit.com/r/CryptoCurrency/comments/8vmcs1/vechainthor_mainnet_launch_gains_backing_from/,PrestigeBMW,0,14


In [12]:
pred = gs.predict(X_train2)

predfinal = pd.Series(data=pred, index=dfraw.index[0:220073])
predfinal.shape
predfinal.name = 'rf2_pred'

[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed:  7.6min finished


In [58]:
dfraw['rf2_pred'] = predfinal
dfraw.tail()

,index,datetime,corpus,permalink,author,subreddit,rowsum,rf2_pred
440140,432955,1.492648e+09,trump epa reconsider oil gas emission rule reuters article usa epa iduskbn,/r/environment/comments/66cqxv/trumps_epa_to_reconsider_oil_and_gas_emissions/,drewiepoodle,1,12,NaN
440141,380070,1.506515e+09,mario draghi european central bank have power regulate bitcoin coindesk mario draghi european central bank have power regulate bitcoin,https://www.reddit.com/r/CryptoCurrency/comments/72jhfa/mario_draghi_european_central_bank_has_no_power/,CryptoCurrency_News,0,19,NaN
440142,96612,1.531306e+09,new htc phone cryptokitties flagship app built htc apps cryptokitties,https://www.reddit.com/r/CryptoCurrency/comments/8xx9ro/new_htc_u12_phone_with_cryptokitties_flagship_app/,twitchtvbitcoinlouie,0,10,NaN
440143,417868,1.513479e+09,josh fox tucker carlson youtube watch rkyqrbkoya,/r/environment/comments/7k9r6g/josh_fox_vs_tucker_carlson/,Kookaburrrra,1,6,NaN
440144,1651,1.544855e+09,noob investor crypto have lot people believe investor market prove lot people shouldn pick individual investment turn get rekt reason stock people elect broker choose investment majority population doesn knowledge time actively research choose good investment reason etf mutual fund index fund exist furthermore lose money market probably suppos...,https://www.reddit.com/r/CryptoCurrency/comments/a6b5cu/noob_investors/,Moneymakessense29,0,61,NaN


In [59]:
y_actual = pd.Series(data=y_train2, index=dfraw.index[0:220073])
dfraw['rf2_errors'] = dfraw['rf2_pred'] - y_actual

In [60]:
dfraw.head()

,index,datetime,corpus,permalink,author,subreddit,rowsum,rf2_pred,rf2_errors
0,439649,1.484256e+09,official say percent california drought powerful storm news news world official say percent california drought powerful storm,/r/environment/comments/5nk84s/officials_say_more_than_40_percent_of_california/,helpmeredditimbored,1,17,1.0,0.0
1,125153,1.527789e+09,opening door everyday user introduce iconomi fiat gateway medium iconominet iconomi fiat gateway,https://www.reddit.com/r/CryptoCurrency/comments/8nim9k/opening_the_doors_for_everyday_users_introducing/,TheAfricaBug,0,13,0.0,0.0
3,568688,1.315469e+09,free chimp cage year inject hiv cruel experiment finally feel sun face dailymail news article lab chimp freed cage fior year inject hiv,/r/environment/comments/k8g39/free_at_last_chimps_caged_for_30_years_and/,Kranky_Old_Dude,1,22,1.0,0.0
5,315118,1.513717e+09,chad satoshi imgur ssbnc,https://www.reddit.com/r/CryptoCurrency/comments/7koez1/chad_satoshi/,Brian_K9,0,3,0.0,0.0
6,102199,1.530581e+09,vechainthor mainnet launch gain backing lbank ambcrypto vechainthor ven mainnet launch gain backing lbank,https://www.reddit.com/r/CryptoCurrency/comments/8vmcs1/vechainthor_mainnet_launch_gains_backing_from/,PrestigeBMW,0,14,0.0,0.0


In [9]:
joblib.dump(gs.best_estimator_, 'randomforestbest2.joblib')

['randomforestbest2.joblib']

In [61]:
dfraw.to_csv('Train2allinfo.csv')

## Random Forest Prediction Errors and Further Pruning?

In [62]:
list_all_hdf5('masterdata.hdf5')

Train1Ytrain
Train1dset
Train2dset
X_Train2
columnsums
columnsums/columnindextodrop
columnsums/columnsumshalf0
columnsums/columnsumshalf1
environment
rowsums
rowsums/columnsumsCrypto
rowsums/columnsumsEnv
rowsums/columnsumsTrain2
rowsums/crypto_dset
rowsums/env_dset
rowsums/rowsumsCrypto
rowsums/rowsumsEnv
rowsums/rowsumsTrain2
rowsums/rowsumshalf0
rowsums/rowsumshalf1
y_train2


In [63]:
with h5py.File("masterdata.hdf5", "r") as z:
    dset = z["rowsums/rowsumsTrain2"]
    rowsums = dset[:]
rowsums = pd.Series(data=rowsums, index=dfraw.index)
rowsums.shape

(344611,)

In [64]:
rowsums.value_counts(ascending=False)[:200]

10     23026
11     19766
13     18746
12     18165
9      17774
14     17121
15     16626
8      16160
7      15613
16     14166
6      13786
17     13253
5      11918
18     10510
19      9344
4       9165
20      7519
21      6589
22      5530
3       5422
23      4801
24      4227
25      3949
26      3629
27      3348
28      3229
29      2783
30      2676
31      2340
32      2171
2       2162
33      1973
34      1732
35      1587
36      1389
37      1227
38      1112
39       993
42       841
40       839
41       812
43       725
44       706
45       642
48       605
46       602
47       578
49       550
50       521
51       507
53       489
52       472
55       449
54       444
56       435
57       412
58       375
61       368
62       356
63       329
60       327
65       325
59       323
1        319
66       309
68       300
64       292
70       288
67       275
72       268
69       254
73       242
74       242
71       233
76       221
75       217
79       206

In [66]:
dfraw.loc[rowsums[lambda x : x < 3].index].sort_values('rf2_errors')

,index,datetime,corpus,permalink,author,subreddit,rowsum,rf2_pred,rf2_errors
200825,471412,1.442267e+09,margaret time sbbh,/r/environment/comments/3kxneb/margaret_your_time_is_up/,lgkffh4f2g1,1,2,0.0,-1.0
90569,563388,1.323247e+09,exterminator charlotte exterminator charlotte,/r/environment/comments/n3ezc/exterminator_charlotte/,hansen80i,1,2,0.0,-1.0
254563,477234,1.434989e+09,blog timesunion holistichealth,/r/environment/comments/3apbj4/unless/,Yogajudi,1,2,0.0,-1.0
26435,563012,1.323774e+09,grow box supercloset,/r/environment/comments/naosx/grow_box/,Supercloset,1,2,0.0,-1.0
62204,460233,1.456366e+09,whore nickname samantha search register bkah aeguy qulb,/r/environment/comments/47f5hj/my_ex_whore_nickname_samantha118_search_can_be/,aybqostrwfnpedg,1,4,0.0,-1.0
119772,559759,1.328771e+09,thinker imgur ufocz png,/r/environment/comments/ph83m/the_thinker/,mrtuckins,1,3,0.0,-1.0
252891,473654,1.439474e+09,thống bụi xulykhithaidaithanh thong bui,/r/environment/comments/3gtwap/hệ_thống_xử_lý_bụi_gỗ/,thuyetle,1,3,0.0,-1.0
252560,563177,1.323958e+09,mug,/r/environment/comments/ndm3i/why_the_100oz_mug/,glockman1727ak47,1,1,0.0,-1.0
164331,586205,1.293450e+09,cf central cfscentral,/r/environment/comments/erxhv/cfs_central/,zzirf,1,1,0.0,-1.0
220184,391695,1.544757e+09,beautifully heartbreaking youtu rjmzxykfbm,/r/environment/comments/a5ycra/beautifully_heartbreaking/,umasr001,1,3,0.0,-1.0


In [67]:
confusion_matrix(y_actual, pred)

array([[136883,    215],
       [  1640,  81335]], dtype=int64)

In [71]:
gs.best_estimator_.feature_importances_.shape

(9466,)

In [74]:
rf2_important_features = pd.read_csv('masterFeaturesVector2.csv')
rf2_important_features.columns = ['rf2_importances','features']
rf2_important_features['rf2_importances'] = gs.best_estimator_.feature_importances_
rf2_important_features.tail()

,rf2_importances,features
9461,0.000002,zrx
9462,0.000009,zuckerberg
9463,0.000000,zug
9464,0.000013,zurich
9465,0.000002,zycrypto


In [75]:
rf2_important_features.sort_values('rf2_importances', ascending=False)

,rf2_importances,features
1516,0.039848,climate
2050,0.036143,crypto
5935,0.034260,org
845,0.031524,bitcoin
2073,0.023739,cryptocurrency
922,0.021253,blockchain
1372,0.016924,change
2942,0.014507,energy
2991,0.014237,environment
1578,0.013792,coin


In [77]:
rf2_important_features[rf2_important_features['rf2_importances'] < 0.0001]['rf2_importances'].value_counts(bins=20)

(-0.0010000998, 4.99e-06]    4399
(4.99e-06, 9.98e-06]         1657
(9.98e-06, 1.5e-05]           847
(1.5e-05, 2e-05]              481
(2e-05, 2.5e-05]              321
(2.5e-05, 2.99e-05]           223
(2.99e-05, 3.49e-05]          187
(3.49e-05, 3.99e-05]          142
(3.99e-05, 4.49e-05]          109
(4.49e-05, 4.99e-05]           80
(4.99e-05, 5.49e-05]           69
(5.99e-05, 6.49e-05]           56
(5.49e-05, 5.99e-05]           45
(6.49e-05, 6.99e-05]           45
(6.99e-05, 7.49e-05]           36
(7.99e-05, 8.49e-05]           33
(8.98e-05, 9.48e-05]           29
(9.48e-05, 9.98e-05]           28
(7.49e-05, 7.99e-05]           27
(8.49e-05, 8.98e-05]           23
Name: rf2_importances, dtype: int64

In [78]:
rf2_important_features[rf2_important_features['rf2_importances'] >= 0.0001]['rf2_importances'].value_counts(bins=20)

(-0.0009397, 0.00209]    537
(0.00209, 0.00407]        41
(0.00407, 0.00606]        17
(0.00606, 0.00805]        14
(0.012, 0.014]             5
(0.00805, 0.01]            3
(0.01, 0.012]              3
(0.014, 0.016]             2
(0.0359, 0.0379]           1
(0.016, 0.018]             1
(0.0379, 0.0398]           1
(0.02, 0.022]              1
(0.022, 0.0239]            1
(0.0299, 0.0319]           1
(0.0339, 0.0359]           1
(0.0239, 0.0259]           0
(0.0259, 0.0279]           0
(0.0279, 0.0299]           0
(0.0319, 0.0339]           0
(0.018, 0.02]              0
Name: rf2_importances, dtype: int64

There are definitely some spam posts still in the corpus, as evidenced by ~50 and ~2000 0 and 1 non-rare word lemma posts. Other posts in the 3 plus subgroups also are not related to either subreddit. Instead of further pruning for edge case types of spam, will try to implement a separate spam vs no spam classification using Multinomial Naive Bayes (in next steps possibly sklearn 0.2's ComplementNB, which is supposed to be able to deal with relatively unbalanced classes). This will be done after the primary subreddit classification with Multinomial Naive Bayes.

## Naive Bayes Multinomial model on Subreddit Classification

In [13]:
# all parameters will be kept at default except possible tweaking of alpha
nb = MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None)
nbmodel = nb.fit(X_train2, y_train2)

In [17]:
pred1 = nbmodel.predict(X_train2[0:110073])
pred2 = nbmodel.predict(X_train2[110073:220073])
pred = np.concatenate((pred1,pred2))
pred.shape

(220073,)

In [20]:
dfraw = pd.read_csv('Train2allinfo.csv')

In [18]:
predfinal = pd.Series(data=pred, index=dfraw.index[0:220073])
predfinal.shape
predfinal.name = 'nb1_pred'

In [22]:
dfraw['nb1_pred'] = predfinal

y_actual = pd.Series(data=y_train2, index=dfraw.index[0:220073])
dfraw['nb1_errors'] = dfraw['nb1_pred'] - y_actual

In [24]:
cross_val_score(nbmodel, X_train2, y_train2)

array([0.98525062, 0.9851139 , 0.9847731 ])

In [37]:
with h5py.File("masterdata.hdf5", "r") as z:
    dset = z["X_Train2"]
    X_test2 = dset[220073:]
    print(X_test2.shape)
    dset = z["y_train2"]
    y_train2 = dset[220073:]
    print(y_train2.shape)

(124538, 9466)
(124538,)


In [29]:
pred = nbmodel.predict(X_test2)
pred.shape

(124538,)

In [38]:
predfinal = pd.Series(data=pred, index=dfraw.index[220073:])
predfinal.shape
predfinal.name = 'nb1_testpred'
dfraw['nb1_testpred'] = predfinal

In [39]:
y_actual = pd.Series(data=y_train2, index=dfraw.index[220073:])
dfraw['nb1_testerrors'] = dfraw['nb1_testpred'] - y_actual

In [42]:
confusion_matrix(y_actual, predfinal)

array([[76780,   695],
       [ 1188, 45875]], dtype=int64)

In [44]:
confusion_matrix(y_actual, predfinal) / 124538

array([[0.61651865, 0.00558063],
       [0.00953926, 0.36836146]])

In [40]:
dfraw.tail()

,Unnamed: 0,index,datetime,corpus,permalink,author,subreddit,rowsum,rf2_pred,rf2_errors,nb1_pred,nb1_errors,nb1_testpred,nb1_testerrors
344606,440140,432955,1.492648e+09,trump epa reconsider oil gas emission rule reuters article usa epa iduskbn,/r/environment/comments/66cqxv/trumps_epa_to_reconsider_oil_and_gas_emissions/,drewiepoodle,1,12,NaN,NaN,NaN,NaN,1.0,0.0
344607,440141,380070,1.506515e+09,mario draghi european central bank have power regulate bitcoin coindesk mario draghi european central bank have power regulate bitcoin,https://www.reddit.com/r/CryptoCurrency/comments/72jhfa/mario_draghi_european_central_bank_has_no_power/,CryptoCurrency_News,0,19,NaN,NaN,NaN,NaN,0.0,0.0
344608,440142,96612,1.531306e+09,new htc phone cryptokitties flagship app built htc apps cryptokitties,https://www.reddit.com/r/CryptoCurrency/comments/8xx9ro/new_htc_u12_phone_with_cryptokitties_flagship_app/,twitchtvbitcoinlouie,0,10,NaN,NaN,NaN,NaN,0.0,0.0
344609,440143,417868,1.513479e+09,josh fox tucker carlson youtube watch rkyqrbkoya,/r/environment/comments/7k9r6g/josh_fox_vs_tucker_carlson/,Kookaburrrra,1,6,NaN,NaN,NaN,NaN,1.0,0.0
344610,440144,1651,1.544855e+09,noob investor crypto have lot people believe investor market prove lot people shouldn pick individual investment turn get rekt reason stock people elect broker choose investment majority population doesn knowledge time actively research choose good investment reason etf mutual fund index fund exist furthermore lose money market probably suppos...,https://www.reddit.com/r/CryptoCurrency/comments/a6b5cu/noob_investors/,Moneymakessense29,0,61,NaN,NaN,NaN,NaN,0.0,0.0


In [45]:
joblib.dump(nbmodel, 'naivebayesbest1.joblib')

['naivebayesbest1.joblib']

In [47]:
dfraw.loc[dfraw['rowsum'] < 3].sort_values('nb1_testerrors')

,Unnamed: 0,index,datetime,corpus,permalink,author,subreddit,rowsum,rf2_pred,rf2_errors,nb1_pred,nb1_errors,nb1_testpred,nb1_testerrors
256966,328117,524824,1.375359e+09,design theblogwander design,/r/environment/comments/1jhe6w/designed_to_move/,ginanicolefontes,1,2,NaN,NaN,NaN,NaN,0.0,-1.0
270590,345448,582833,1.297733e+09,prescience sue bit fudb,/r/environment/comments/flc6d/the_prescience_of_dr_suess/,ecocommish,1,2,NaN,NaN,NaN,NaN,0.0,-1.0
274420,350382,551268,1.340620e+09,bank job banksjobs,/r/environment/comments/vk6r1/bank_jobs/,procall01,1,2,NaN,NaN,NaN,NaN,0.0,-1.0
224282,286345,540543,1.355584e+09,hotel seychelles hotelsseychelles,/r/environment/comments/14w2cs/hotels_seychelles/,jenrviken,1,2,NaN,NaN,NaN,NaN,0.0,-1.0
329436,420823,398029,1.538623e+09,redd jpg,/r/environment/comments/9l6dat/do_it/,participatory1,1,2,NaN,NaN,NaN,NaN,0.0,-1.0
329528,420941,562350,1.325211e+09,peep window,/r/environment/comments/nvbzs/a_peep_through_the_window/,enriakriti,1,2,NaN,NaN,NaN,NaN,0.0,-1.0
337966,431715,547284,1.346077e+09,slotmaschine kostenlos spielen einarmigerbandit,/r/environment/comments/ywbw0/slotmaschine_kostenlos_spielen/,asiawopro11,1,2,NaN,NaN,NaN,NaN,0.0,-1.0
303550,387633,558090,1.331054e+09,time archersofokcular time,/r/environment/comments/qk2xa/once_upon_a_time/,okcularvillagevoice,1,2,NaN,NaN,NaN,NaN,0.0,-1.0
243969,311438,549736,1.343161e+09,betsson bonus betsson casino,/r/environment/comments/x2ukx/betsson_bonus/,procall01,1,2,NaN,NaN,NaN,NaN,0.0,-1.0
248865,317741,445530,1.476440e+09,savecal facebook facebook savecal,/r/environment/comments/57ezqa/savecal_on_facebook/,Savecalusa,1,2,NaN,NaN,NaN,NaN,0.0,-1.0


## Support Vector Machine Model

In [3]:
with h5py.File("masterdata.hdf5", "r") as z:
    dset = z["X_Train2"]
    X_train2svc = dset[0:2273]
    print(X_train2svc.shape)
    dset = z["y_train2"]
    y_train2svc = dset[0:2273]
    print(y_train2svc.shape)

(2273, 9466)
(2273,)


In [5]:
scaler = RobustScaler()
X_train2svc = scaler.fit_transform(X_train2svc)

In [9]:
svc = svm.SVC(C = 2,
              kernel = 'poly')

In [10]:
# Can't run on normal 220073 Train2 dataset due to computer memory limitations, 
# instead used first 110073 subset of Train2
svc.fit(X_train2svc, y_train2svc)

SVC(C=2, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [ ]:
pred = svc.predict(X_train2svc)

In [14]:
dfraw = pd.read_csv('Train2allinfo.csv')

In [15]:
predfinal = pd.Series(data=pred, index=dfraw.index[0:2273])
predfinal.shape
predfinal.name = 'svc_pred'

In [17]:
dfraw['svc_pred'] = predfinal

y_actual = pd.Series(data=y_train2svc, index=dfraw.index[0:2273])
dfraw['svc_errors'] = dfraw['svc_pred'] - y_actual

In [19]:
confusion_matrix(y_train2svc, predfinal)  # Not Good! Only predictions for Cryptocurrency

array([[1421,    0],
       [ 852,    0]], dtype=int64)

In [20]:
predfinal.value_counts()

0    2273
Name: svc_pred, dtype: int64

In [48]:
# Not worth adding svc predictions to main records, something is wrong or calibration is way off
# dfraw.to_csv('Train2allinfo.csv')